# TIF360 Project

Main source: https://www.kaggle.com/code/rmonge/predicting-molecule-properties-based-on-its-smiles/notebook

### Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time as time

import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.nn import Linear, BatchNorm1d as BatchNorm
import torch.nn.functional as F

from sklearn.metrics import r2_score
import utility_functions as uf

In [2]:
print("cuda available:", torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", "cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

cuda available: True
device: cuda
NVIDIA GeForce RTX 3080


### Load data

Load smiles data and targets

In [3]:
df = pd.read_csv("../data/smiles_and_targets.csv")
print(np.shape(df))

(132820, 21)


Choose mode: Which features to include

In [4]:
# singles
# mode = "rdkit_descriptors"
# mode = "morgan_fingerprints"
# mode = "mordred_descriptors"

# combinations
# mode = "rdkit+morgan"
mode = "mordred+morgan"

# excessive
# mode = "rdkit+mordred"
# mode = "rdkit+mordred+morgan"

Load the features

In [5]:
features = uf.load_molecular_features(mode)
print("features:", np.shape(features))

features: (132820, 3033)


In [6]:
def create_scale_and_split_dataset(features:np.ndarray, targets:np.ndarray, val_share, test_share):
    
    num_samples = features.shape[0]
    print("num_samples:", num_samples)
    train_indices, val_indices, test_indices = uf.get_data_split_indices(num_samples, val_share, test_share)
    
    X_train, y_train = features[train_indices], targets[train_indices]
    X_val, y_val = features[val_indices], targets[val_indices]
    X_test, y_test = features[test_indices], targets[test_indices]
    
    X_train, X_val, X_test, scaler_features = uf.scale_features(X_train, X_val, X_test)
    y_train, y_val, y_test, scaler_targets = uf.scale_targets(y_train, y_val, y_test)
    
    scalers = {"features": scaler_features, "targets": scaler_targets}
    
    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float, device=device),
                               torch.tensor(y_train, dtype=torch.float, device=device))
    val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float, device=device),   
                             torch.tensor(y_val, dtype=torch.float, device=device))
    test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float, device=device),
                              torch.tensor(y_test, dtype=torch.float, device=device))

    return train_data, val_data, test_data, scalers

def create_data_loaders(train_data, val_data, test_data, batch_size):
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader  

Load data

In [7]:
print("...Loading data...")
properties_names = ['A', 'B', 'C', 'mu', 'alfa', 'homo', 'lumo', 'gap', 'R²', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']
x_smiles = df.smiles.values
targets = df.loc[:, properties_names].values

train_data, val_data, test_data, scalers = create_scale_and_split_dataset(features, targets, 0.15, 0.2)
train_loader, val_loader, test_loader = create_data_loaders(train_data, val_data, test_data, 64)
print("...Data loading done...")

...Loading data...
num_samples: 132820
...Data loading done...


### Dense network 

#### Model for all targets at once

Define model

In [8]:
class DNN(torch.nn.Module):
    def __init__(self, hidden_channels, feature_dim, target_dim):
        super().__init__()
        torch.manual_seed(12345)
        
        self.input_norm = BatchNorm(feature_dim)
        self.lin1 = Linear(feature_dim, hidden_channels)
        self.lin2 = Linear(hidden_channels, 512)
        self.lin3 = Linear(512, target_dim)

    def forward(self, x): 
                
        x = self.input_norm(x)
        x = self.lin1(x)
        x = F.leaky_relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        
        x = self.lin2(x)
        x = F.leaky_relu(x)

        x = self.lin3(x)
 
        return x

Train DNN

In [12]:
def train(model, batch):
      features = batch[:][0]
      targets = batch[:][1]
      
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(features).to(device)  # Perform a single forward pass.

      loss = criterion(out, targets) 

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      
      return loss

def test(model, data):
      for i, batch in enumerate(data):
            features = batch[:][0]
            targets = batch[:][1].cpu()
            
            model.eval()
            out = model(features).cpu()
            
            # Caculate R2 for each target
            for target_idx in range(target_dim):
                  if target_idx != 0:
                        r2_score_var = np.vstack((r2_score_var, r2_score(targets[:,target_idx].detach().numpy(), 
                                                          out[:,target_idx].detach().numpy())))
                  else:
                        r2_score_var = np.array([r2_score(targets[:,target_idx].detach().numpy(),
                                                          out[:,target_idx].detach().numpy())])        
            all_r2 = np.hstack((all_r2, r2_score_var)) if i != 0 else r2_score_var
                  
            loss = float(criterion(out, targets).detach().numpy())
            all_loss = np.hstack((all_loss, loss)) if i != 0 else np.array(loss)

      average_test_r2 = np.mean(all_r2, axis=1)
      average_test_loss = np.mean(all_loss)
      
      return average_test_r2, average_test_loss

def early_stopping(val_losses, patience): # returns True if there is no improvement in val_loss
      if len(val_losses) < patience:
            return False
      else:
            best_loss = np.min(val_losses)
            current_loss = val_losses[-1]
            
            if current_loss > best_loss:
                  return True
            else:
                  return False

feature_dim = train_data[:][0].shape[1]
target_dim = train_data[:][1].shape[1]

model = DNN(hidden_channels=1024, feature_dim=feature_dim, target_dim=target_dim).to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# Decay for learning rate
decay_rate = 0.94
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

criterion = torch.nn.MSELoss().to(device)

train_params, _ = uf.get_num_parameters(model)
print(f"Trainable parameters: {train_params:,}")

# Vectors to append accuracy to:
train_r2 = []
train_loss = []
test_r2 = []
test_loss = []
val_r2 = []
val_loss = []

n_epochs = 100
print_every_N_epochs = False
N = 10 # print R2 every N epochs

epoch_times = []
train_times = []
test_times = []
print()
print("...Starting training...")
print("Device used:", device)

val_losses_epoch = [] # for early stopping
patience = 5 # how many epochs to wait for the val loss to improve
best_val_loss = np.inf  
epochs_without_improvement = 0

for epoch in np.arange(1, n_epochs+1):
      epoch_start = time.time()
      losses = []
      train_start = time.time()
      for batch in train_loader:
            loss = train(model, batch)
            losses.append(loss.cpu().detach().numpy())  
      # Compute validation loss
      model.eval()
      val_losses = []
      for batch in val_loader:
            features = batch[:][0]
            targets = batch[:][1]
            out = model(features)
            val_losses.append(criterion(out, targets).cpu().detach().numpy()) 

      val_loss_epoch = np.mean(val_losses)
      val_losses_epoch.append(val_loss_epoch)
      print(f"Epoch: {epoch:02d} | Train Loss: {np.mean(losses):.5f} | Validation Loss: {val_loss_epoch:.5f}")
      train_end = time.time()
      train_times.append(train_end - train_start)
      lr_scheduler.step() # Decay to learning rate
      
      # check for early stopping
      if early_stopping(val_losses_epoch, patience) and epoch > patience:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                  print(f"Early stopping at epoch {epoch} after {patience} epochs without improvement.")
                  break
      else:
            epochs_without_improvement = 0
            best_val_loss = val_loss_epoch
            # save the model
            torch.save(model.state_dict(), 'best_features_model.pt')
            
      if print_every_N_epochs and (epoch % N == 0 or epoch == 1) and epoch != n_epochs:
            test_start = time.time()
                
            r2_temp_val, loss_temp_val = test(model, val_loader)
            val_r2.append(r2_temp_val)
            val_loss.append(loss_temp_val)
            
            print(f'Validation R2: {r2_temp_val}')
            print(f"Validation loss: {loss_temp_val}")
            test_end = time.time()
            test_times.append(test_end - test_start)
            
            model.load_state_dict(torch.load('best_features_model.pt'))
            
      epoch_end = time.time()
      epoch_times.append(epoch_end - epoch_start)
      
print("...Training done...")
print("...Calculating final results...")
model.load_state_dict(torch.load('best_features_model.pt'))

r2_temp_train, loss_temp_train = test(model, train_loader) 
train_r2.append(r2_temp_train)
train_loss.append(loss_temp_train)

r2_temp_val, loss_temp_val = test(model, val_loader)
val_r2.append(r2_temp_val)
val_loss.append(loss_temp_val)

r2_temp_test, loss_temp_test = test(model, test_loader)
test_r2.append(r2_temp_test)
test_loss.append(loss_temp_test)

print("====================================================")
print("Final training R2:", train_r2[-1])
print("Average final training R2: ", np.mean(train_r2[-1]))
print("Final training loss:", train_loss[-1])

print("Final validation R2:", val_r2[-1])
print("Average validation R2: ", np.mean(val_r2[-1]))
print("Final validation loss:", val_loss[-1])

print("Final test R2:", test_r2[-1])
print("Average final test R2: ", np.mean(test_r2[-1]))
print("Final test loss:", test_loss[-1])
            

Trainable parameters: 3,645,377

...Starting training...
Device used: cuda
Epoch: 01 | Train Loss: 0.12394 | Validation Loss: 0.08419
Epoch: 02 | Train Loss: 0.10395 | Validation Loss: 0.07989
Epoch: 03 | Train Loss: 0.09852 | Validation Loss: 0.07871
Epoch: 04 | Train Loss: 0.09432 | Validation Loss: 0.07257
Epoch: 05 | Train Loss: 0.09179 | Validation Loss: 0.07492
Epoch: 06 | Train Loss: 0.08955 | Validation Loss: 0.06772
Epoch: 07 | Train Loss: 0.08726 | Validation Loss: 0.06839
Epoch: 08 | Train Loss: 0.08494 | Validation Loss: 0.06816
Epoch: 09 | Train Loss: 0.08486 | Validation Loss: 0.06585
Epoch: 10 | Train Loss: 0.08362 | Validation Loss: 0.06643
Epoch: 11 | Train Loss: 0.08239 | Validation Loss: 0.06338
Epoch: 12 | Train Loss: 0.08128 | Validation Loss: 0.06313
Epoch: 13 | Train Loss: 0.08056 | Validation Loss: 0.06261
Epoch: 14 | Train Loss: 0.07953 | Validation Loss: 0.06121
Epoch: 15 | Train Loss: 0.07820 | Validation Loss: 0.06009
Epoch: 16 | Train Loss: 0.07781 | Valida

In [10]:
print("Device used:", device)
print()
print(f"Total number of epochs: {len(epoch_times)}")
print(f"Total training time: {np.sum(epoch_times)/60:.2f} minutes")
print(f"Total time in training: {np.sum(train_times)/60:.2f} minutes")
print(f"Total time in testing: {np.sum(test_times)/60:.2f} minutes")
print()
print(f"Average epoch time: {np.mean(epoch_times):.1f} seconds")
print(f"Average time in training: {np.mean(train_times):.1f} seconds")
print(f"Average time in testing: {np.mean(test_times):.1f} seconds")

Device used: cuda

Total number of epochs: 48
Total training time: 2.43 minutes
Total time in training: 2.48 minutes
Total time in testing: 0.00 minutes

Average epoch time: 3.0 seconds
Average time in training: 3.0 seconds
Average time in testing: nan seconds


c:\Users\jespe\anaconda3\envs\proj_env\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jespe\anaconda3\envs\proj_env\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


#### Model for just one target

Train DNN

In [11]:
feature_dim = train_data[:][0].shape[1]
target_dim = 1

def train(batch, target_idx):
      features = batch[:][0]
      targets = batch[:][1][:,target_idx].unsqueeze(-1)
      
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(features).to(device)  # Perform a single forward pass.

      loss = criterion(out, targets) 

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      
      return loss

def test(data, target_idx):
      for i, batch in enumerate(data):
            features = batch[:][0]
            targets = batch[:][1][:,target_idx].cpu().unsqueeze(-1)
            
            model.eval()
            out = model(features).cpu()
            
            # Caculate R2    
            r2_score_var = np.array([r2_score(targets.detach().numpy(), out.detach().numpy())])        
            all_r2 = np.hstack((all_r2, r2_score_var)) if i != 0 else r2_score_var
                  
            loss = float(criterion(out, targets).detach().numpy())
            all_loss = np.hstack((all_loss, loss)) if i != 0 else np.array(loss)

      average_test_r2 = np.mean(all_r2)
      average_test_loss = np.mean(all_loss)
      
      return average_test_r2, average_test_loss

num_targets = train_data[:][1].shape[1]
start_time = time.time()
for target_index in range(num_targets):
      print("Target index:", target_index)

      model = DNN(hidden_channels=1024, feature_dim=feature_dim, target_dim=target_dim).to(device) 
      optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
      criterion = torch.nn.MSELoss().to(device)
      
      # Decay for learning rate
      decayRate = 0.9
      lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

      # Vectors to append accuracy to:
      train_r2 = []
      train_loss = []
      test_r2 = []
      test_loss = []
      val_r2 = []
      val_loss = []

      n_epochs = 10

      print("Device used:", device)

      for epoch in np.arange(1, n_epochs+1):
            losses = []
            for data in train_loader:
                  loss = train(data, target_index)
                  losses.append(loss.cpu().detach().numpy())  
            # Compute validation loss
            model.eval()
            val_losses = []
            for batch in val_loader:
                  features = batch[:][0]
                  targets = batch[:][1][:, target_index].unsqueeze(-1)
                  out = model(features)
                  val_losses.append(criterion(out, targets).cpu().detach().numpy())
            print(f"Epoch: {epoch:02d} | Train Loss: {np.mean(losses):.5f} | Validation Loss: {np.mean(val_losses):.5f}")
            
            lr_scheduler.step() # Decay to learning rate
                  
            if epoch == n_epochs:         # calculate results of training: test on all data
                  test_start = time.time()
                  r2_temp_train, loss_temp_train = test(train_loader, target_index) 
                  train_r2.append(r2_temp_train)
                  train_loss.append(loss_temp_train)
                  
                  r2_temp_val, loss_temp_val = test(val_loader, target_index)
                  val_r2.append(r2_temp_val)
                  val_loss.append(loss_temp_val)
                  
                  r2_temp_test, loss_temp_test = test(test_loader, target_index)
                  test_r2.append(r2_temp_test)
                  test_loss.append(loss_temp_test)

                  print("====================================================")
                  print("Final training R2:", train_r2[-1])
                  print("Average final training R2: ", np.mean(train_r2[-1]))
                  print("Final training loss:", train_loss[-1])
                  print()
                  print("Final validation R2:", val_r2[-1])
                  print("Average validation test R2: ", np.mean(val_r2[-1]))
                  print("Final validation loss:", val_loss[-1])
                  print()
                  print("Final test R2:", test_r2[-1])
                  print("Average final test R2: ", np.mean(test_r2[-1]))
                  print("Final test loss:", test_loss[-1])
                  print("====================================================")
                                    
print("...Done...")
end_time = time.time()
print(f"Time taken: {(end_time - start_time)/60} minutes")
print(f"Average time per target: {(end_time - start_time)/(num_targets*60)} minutes")

Target index: 0
Device used: cuda
Epoch: 01 | Train Loss: 0.20794 | Validation Loss: 0.18548
Epoch: 02 | Train Loss: 0.13513 | Validation Loss: 0.16727
Epoch: 03 | Train Loss: 0.11163 | Validation Loss: 0.16146
Epoch: 04 | Train Loss: 0.09687 | Validation Loss: 0.17716
Epoch: 05 | Train Loss: 0.08728 | Validation Loss: 0.16383
Epoch: 06 | Train Loss: 0.08045 | Validation Loss: 0.15189


KeyboardInterrupt: 